In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

In [2]:
import sys
!{sys.executable} -m pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 2.2 MB/s eta 0:00:0000:0100:01


In [5]:
import sys
!{sys.executable} -m pip install scikit-optimize

In [6]:
import sys
!{sys.executable} -m pip install wandb

In [7]:
import os
os.environ['WANDB_API_KEY'] = '09166f4249028d9a37cd7a0c4efb38343a046802'

In [8]:
import wandb
wandb.init(project="Exoplanet", entity="adrita_khan",save_code=True,name='XGBoost Classifier',job_type='analysis',reinit=True)

wandb: Currently logged in as: adrita_khan. Use `wandb login --relogin` to force relogin


In [9]:
import numpy as np

In [10]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [11]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [12]:
from skopt import BayesSearchCV
import xgboost as xgb

In [13]:
# Create an instance of the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(random_state=43,n_jobs=-1)

In [14]:
# Define the parameter grid for XGBoost
params = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.2),
    'n_estimators': (400, 500),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1)
}

In [15]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [16]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(xgb_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [ ]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

In [ ]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

In [ ]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [ ]:
wandb.finish()